In this last notebook, I want to cover a two more topics. The most essential is the Instrument and everything associated which handles different detector frequencies. This will also cover how I use PySM3 to generate simulations and how I use matplotlib to produce figures.

NOTE TO SELF: IS THIS NEEDED?

# Matplotlib Executor: Seeing Results

In this last example, I want to show a few things that appear in Analysis stages, where Matplotlib is used. Some points to watch for:
- The assets in need to change asset labels
- To avoid conflict with Matplotlib, the AssetHandler is indirect
- I override the number of simulations to process, producing a limited number of figures

These Executors tend to be some of the most finicky. This is where I tend to depart from my goals of having all parameters in the configuration files, simply because there are so many configuration options and because the combination of healpy and matplotlib need a lot of adjustments.

# Configuration YAML

YAML set up for G_, but ... I want to be done and do something else (and it's SUPER LONG ALREADY)
```yaml
  figure:
    assets_out:
      compare_fig:
        handler: Mover
        path_template: "{root}/{dataset}/{stage}/{split}/{sim}/compare_fig.png"
    assets_in:
      cmb_raw: {stage: ps2map, orig_name: cmb_map}
      cmb_prep: {stage: prep, orig_name: cmb_map}
    dir_name: E_Figure
    splits: valid
    make_stage_log: false
```

First, I get my stage set up. The output is going to be a figure. To make this, I need both the raw map and the preprocessed map.

In [ ]:
print(OmegaConf.to_yaml(cfg.pipeline.figure))

Notice that the first in `assets_in` is `cmb_raw`, which draws from the `ps2map` stage.

In [ ]:
print(OmegaConf.to_yaml(cfg.pipeline.ps2map.assets_out))

In that stage, the only output asset is "cmb_map". Both the raw CMB and preprocessed CMB were given the name "cmb_map". Instead of changing those names to something particular (which comes with several headaches), I use the `orig_name` tag. This is a cue to look for `cmb_map` in the original stage.

## Executor

Now I can 

In [ ]:
from cmbml.core.asset_handlers import Mover

class FigureExecutor(BaseStageExecutor):
    def __init__(self, cfg) -> None:
        super().__init__(cfg, stage_str="figure")

        self.out_cmb_figure: Asset = self.assets_out["cmb_map_render"]
        out_cmb_figure_handler: Mover

        self.in_cmb_map_sim: Asset = self.assets_in["cmb_map_sim"]
        self.in_cmb_map_pred: Asset = self.assets_in["cmb_map_pred"]
        in_maps_handler: HealpyMap

        if self.override_sim_nums is None:
            logger.warning("No particular sim indices specified. Outputs will be produced for all. This is not recommended.")
        self.min_max = self.get_plot_min_max()
        self.fig_model_name = cfg.fig_model_name
